In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import pickle as pkl
import numpy as np
import random
import seaborn as sns


In [ ]:
# Load the data
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pkl.load(f)
    data_no_advertisers = [
        {
            k: v for k, v in d.items() if k != 'advertisers'
        } for d in data
    ]
    all_advertisers = [d['advertisers'] for d in data]
    
    # Create a dataframe
    df = pd.DataFrame.from_records(data_no_advertisers, index=range(len(data)))
    return df, all_advertisers


In [ ]:
df, all_advertisers = load_data('ga_results5.pkl')

In [ ]:
def tau(e, coeffs):
    tau_val = 0
    for i, coeff in enumerate(coeffs):
        tau_val += coeff * (e**i)
    return tau_val

def plot_advertisers(auction_output, ad_scaler=1, ext_scaler=1):
    # plot all advertisers and best line
    advertisers = auction_output['advertisers']
    tau_coeffs = auction_output['tau']
    v = [ad[0]*ad_scaler for advertisers_set in advertisers for ad in advertisers_set]
    e = [ad[1]*ext_scaler for advertisers_set in advertisers for ad in advertisers_set]
    min_e, max_e = min(e), max(e)*1.1
    division_size = int(((2*max_e) - min_e) / 100)
    tau_x = [min_e + i*division_size for i in range(100)]
    tau_y = [tau(t, tau_coeffs) for t in tau_x]

    fig, axs = plt.subplots(1, 1, figsize=(10, 5))

    axs.scatter(e, v, alpha=0.5)
    axs.plot(tau_x, tau_y, color='red')
    axs.set_ylabel('Advertiser Value')
    axs.set_xlabel('Externality Value')
    axs.set_title('Social Welfare Values vs Advertiser Values with Best Tau Line')

    # set x and y limits
    axs.set_xlim(min_e, max_e)
    axs.set_ylim(0, max(v)*1.1)
    fig.show()
    
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=256):
    """Create a new colormap from a subset (minval->maxval) of an existing cmap."""
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        f'trunc({cmap.name},{minval:.2f},{maxval:.2f})',
        cmap(np.linspace(minval, maxval, n))
    )
    return new_cmap

# Plotting the Data

Desired plots are:
- ~~Ad. welfare from Ext. welfare for each polynomial degree~~
- ~~triplot of change in welfare vs externality cost~~
- ~~example of tau with advertisers~~
- ~~tau from different k and poly but same lambda~~
- ~~histogram distributions of welfare values~~

In [ ]:
# Plot ad welfare vs ext welfare
df_k_1 = df[df['k'] == 1]

sns.set_style('whitegrid')
# Define a mapping from polynomial_degree to a named colormap
cmap_dict = {
    1: 'Blues',
    2: 'Reds',
    3: 'Greens'
}

original_blues = plt.cm.Blues
short_blues = truncate_colormap(original_blues, minval=0.5, maxval=1.0)


plt.figure(figsize=(8,6))
# We will keep track of the min and max of externality_cost_per_impression
# so that each scatter uses the same color scale
vmin = df['externality_cost_per_impression'].min()
vmax = df['externality_cost_per_impression'].max()

for poly in [1]:#, 2, 3]:
    subset = df_k_1[df_k_1['polynomial_degree'] == poly]
    
    # Plot each subset with its own colormap
    scatter = plt.scatter(
        x=subset['w_coll_ext'],
        y=subset['w_coll_adv'],
        c=subset['externality_cost_per_impression'],
        cmap=short_blues,
        alpha=0.7,  # slightly transparent
        vmin=vmin,  # ensures consistent color scale across subsets
        vmax=vmax,
        label=f'polynomial degree={poly}'
    )
# A single colorbar for all points
cbar = plt.colorbar(scatter, label='Externality Cost Per Impression')

plt.xlabel('Externality Welfare')
plt.ylabel('Advertiser Welfare')
plt.title('Advertiser vs. Externality Welfare')
# plt.legend(title='Tau Polynomial Degree')
# plt.tight_layout()
plt.show()

In [ ]:
df_k_1['w_adv_change'] = (df_k_1['w_coll_adv'] - df_k_1['w_vcg_adv'])/1000
df_k_1['w_ext_change'] = (df_k_1['w_coll_ext'] - df_k_1['w_vcg_ext'])/1000
df_k_1['w_tot_change'] = (df_k_1['w_coll_tot'] - df_k_1['w_vcg_tot'])/1000

y_labels = ['Change in Advertiser Welfare (thousands)', 'Change in Externality Welfare (thousands)', 'Change in Total Welfare (thousands)']

# three subplots for each change in welfare
fig, axs = plt.subplots(1, 3, figsize=(14, 5))

for i, col in enumerate(['w_adv_change', 'w_ext_change', 'w_tot_change']):
    for poly in [1]:#, 2, 3]:
        subset = df_k_1[df_k_1['polynomial_degree'] == poly]
        subset.sort_values(by='externality_cost_per_impression', inplace=True)
        
        # Plot each subset with its own colormap
        scatter = axs[i].plot(
            subset['externality_cost_per_impression'],
            subset[col],
            marker='o',
            alpha=0.7,  # slightly transparent
            label=f'polynomial degree={poly}',
        )
    axs[i].set_ylabel(y_labels[i])
    axs[i].set_xlabel('Externality Cost Per Impression')

# set figure title
fig.suptitle('Change in Welfare vs. Externality Cost')

# --- Deduplicate handles & labels across subplots ---
# 1. Collect from each axis
handles, labels = [], []
for ax in axs:
    h, l = ax.get_legend_handles_labels()
    handles.extend(h)
    labels.extend(l)

# 2. Remove duplicates by zipping into a dict
by_label = dict(zip(labels, handles))

# 3. Create one legend for the figure, using unique labels
# fig.legend(by_label.values(), by_label.keys(), 
#            title='Tau Polynomial Degree', 
#            loc='lower right')
# fig.tight_layout()
fig.show()

In [ ]:
df_for_example_tau = df_k_1[df_k_1['externality_cost_per_impression'] == 0.01]
df_indices = df_for_example_tau.index

# plot advertisers and tau line for each row

titles = ['Affine Tau', 'Quadratic Tau', 'Cubic Tau']
titles = [f'{title} (W={round(df_for_example_tau["w_coll_tot"].iloc[i])})' for i, title in enumerate(titles)]

# three subplots for each change in welfare
fig, axs = plt.subplots(1, 3, figsize=(14, 5))

for i in range(3):
    subset = df_for_example_tau[df_for_example_tau['polynomial_degree'] == i+1]
    advertisers = all_advertisers[df_indices[i]]

    tau_coeffs = subset['tau'].iloc[0]
    
    v = [ad[0] for advertisers_set in advertisers for ad in advertisers_set]
    e = [ad[1] for advertisers_set in advertisers for ad in advertisers_set]
    min_e, max_e = min(e), max(e)*1.1
    division_size = int(((2*max_e) - min_e) / 100)
    tau_x = [min_e + i*division_size for i in range(100)]
    tau_y = [tau(t, tau_coeffs) for t in tau_x]

    axs[i].scatter(e, v, alpha=0.5)
    axs[i].plot(tau_x, tau_y, color='red')
    # axs[i].set_ylabel('Advertiser Value')
    # axs[i].set_xlabel('Externality Value')
    axs[i].set_title(titles[i])

    # set x and y limits
    axs[i].set_xlim(min_e, max_e)
    axs[i].set_ylim(0, max(v)*1.1)
    

# set figure title
fig.suptitle('Optimal Affine, Quadratic and Cubic Tau Functions (ζ=0.01)')
fig.supxlabel('Externality Value')
fig.supylabel('Advertiser Value')
fig.tight_layout()
fig.show()




In [ ]:
# load k-sweep data
df, all_advertisers = load_data('ga_results_k_2.pkl')
print(df.shape)

In [ ]:

cmap = {1: 'blue', 2: 'red',5: 'orange'}
full_advertisers_set = [ad for advertisers_set in all_advertisers for ad in advertisers_set]

v = [ad[0] for advertisers_set in advertisers for ad in advertisers_set]
e = [ad[1] for advertisers_set in advertisers for ad in advertisers_set]
min_e, max_e = min(e), max(e)*1.1
division_size = int(((2*max_e) - min_e) / 100)
tau_x = [min_e + i*division_size for i in range(100)]

plt.scatter(e, v, alpha=0.5)

for i in range(9):

    tau_coeffs = df['tau'].iloc[i]
    
    tau_y = [tau(t, tau_coeffs) for t in tau_x]

    plt.plot(tau_x, tau_y, color=cmap[df['k'].iloc[i]])

# set x and y limits
plt.xlim(min_e, max_e)
plt.ylim(0, max(v)*1.1)

# Create one custom legend entry per color (k-value)
blue_line  = mlines.Line2D([], [], color='blue',  label='k=1')
red_line   = mlines.Line2D([], [], color='red',   label='k=2')
green_line = mlines.Line2D([], [], color='orange', label='k=5')

plt.legend(handles=[blue_line, red_line, green_line], 
           loc='best', title="Value of k")
    

# set figure title
plt.title('Optimal Tau Functions for Varied K (ζ=0.01)')
plt.xlabel('Externality Value')
plt.ylabel('Advertiser Value')
plt.tight_layout()
plt.show()